In [59]:
import torch
from torch import nn
from torch.utils.data import DataLoader
import pandas as pd

In [60]:
# Read Data
spectrum_train = pd.read_excel('spectrum_train.xlsx')
spectrum_test = pd.read_excel('spectrum_valid.xlsx')
temp_train = pd.read_excel('temp_train.xlsx')
temp_test = pd.read_excel('temp_valid.xlsx')

In [63]:
class ThermDataset(torch.utils.data.Dataset):
    def __init__(self, X, y):
        if not torch.is_tensor(X) and not torch.is_tensor(y):
            self.X = torch.from_numpy(X)
            self.y = torch.from_numpy(y)

    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, i):
        return self.X[i], self.y[i]

In [64]:
class NN(nn.Module):
    def __init__(self):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(66, 100),
            nn.ReLU(),
            nn.Linear(100, 100),
            nn.ReLU(),
            nn.Linear(100, 11)
        )

    def forward(self, x):
        return self.layers(x)

In [65]:
X = spectrum_train.to_numpy()
y = temp_train.to_numpy()

In [66]:
dataset = ThermDataset(X, y)
trainloader = torch.utils.data.DataLoader(dataset, batch_size=10, shuffle=True, num_workers=1)

In [67]:
model = NN()

In [70]:
loss_fn = nn.L1Loss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

for epoch in range(0, 5):
    print(f"Starting epoch {epoch+1}")
    current_loss = 0.0

    for i, data in enumerate(trainloader, 0):
        inputs, targets = data
        inuts, targets = datainputs, targets = inputs.float(), targets.float()

        optimizer.zero_grad()

        outputs = model(inputs)

        loss = loss_fn(outputs, targets)

        loss.backward()

        optimizer.step()

        current_loss += loss.item()
        if i % 10 == 0:
            print('Loss after miit-batch %5d: %.3f' % (i + 1, current_loss / 500))
            current_loss = 0.0


Starting epoch 1


RuntimeError: DataLoader worker (pid(s) 13696) exited unexpectedly